In [1]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from pulp import *
import itertools
MEPS = 1.0e-8

In [2]:
def edge_coloring(G):
  delta = max([G.degree(v) for v in G.nodes()])
  k = delta
  solved = False

  while not(solved) and k <= delta+1:
    prob = LpProblem(name='Edge_Coloring_by_PuLP', sense=LpMinimize)

    x = {(e,i): LpVariable('x'+str(e)+str(',')+str(i),lowBound=0,cat='Binary')
         for e in G.edges() for i in range(k)}

    prob += 0
    for e in G.edges:
      prob += lpSum(x[(e,i)] for i in range(k)) == 1

    for (u,i) in itertools.product(G.nodes(), range(k)):
      el = [tuple(sorted((u,v))) for v in G.neighbors(u)]
      prob += lpSum(x[(e,i)] for e in el) <= 1

    prob.solve()
    if LpStatus[prob.status] == 'Optimal':
      solved = True
    else:
      k += 1

  if solved:
    print('Edge '+str(k)+' coloring found:')
    coloring = {i: [e for e in G.edges if x[(e,i)].varValue > MEPS] for i in range(k)}
    print(coloring)
  else:
    print("Error:")

In [3]:
G=nx.Graph()
elist = [(1,2),(3,1),(1,4),(4,2),(2,5),(4,3),(3,5)]
G.add_edges_from(elist)
edge_coloring(G)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/d8d2c10c40304cfb99a35eb87b5abf10-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/d8d2c10c40304cfb99a35eb87b5abf10-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 134 RHS
At line 157 BOUNDS
At line 180 ENDATA
Problem MODEL has 22 rows, 22 columns and 63 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 22 rows, 21 columns (21 integer (21 of which binary)) and 63 elements
Cbc0038I Initial state - 13 integers unsatisfied sum - 5
Cbc0038I Pass   1: suminf.    5.00000 (13) obj. 0 iterations 5
Cbc0038I Pass   2: suminf.    5.0

In [5]:
K5=nx.complete_graph(5)
edge_coloring(K5)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/2c04a7484b734e26bb6eda7190a7d2b7-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/2c04a7484b734e26bb6eda7190a7d2b7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 35 COLUMNS
At line 237 RHS
At line 268 BOUNDS
At line 310 ENDATA
Problem MODEL has 30 rows, 41 columns and 120 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 30 rows, 40 columns (40 integer (40 of which binary)) and 120 elements
Cbc0038I Initial state - 12 integers unsatisfied sum - 6
Cbc0038I Pass   1: suminf.    6.00000 (12) obj. 0 iterations 11
Cbc0038I Pass   2: suminf.    

In [6]:
import networkx as nx
import collections

def welsh_powell(G):
  delta = max([G.degree(v) for v in G.nodes()])
  sv = collections.deque([v for (v,d) in 
       sorted(G.degree(), key=lambda x: x[1])])
  nbcols = {v: set([]) for v in sv}

  cset = set(range(delta+1))
  c = min(cset)
  u = sv.pop()
  colors = {u:c}
  for v in G.neighbors(u):
    nbcols[v].update({c})
  cls = {c}
  while len(sv) > 0:
    u = sv.pop()
    c = min(cset-nbcols[u])
    colors[u] = c
    cls.update({c})
    for v in G.neighbors(u):
      nbcols[v].update({c})

  print('Node '+str(len(cls))+' coloring found.')

In [7]:
welsh_powell(G)

Node 3 coloring found.


In [8]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from pulp import *
import itertools
MEPS = 1.0e-8

In [11]:
def node_coloring_by_PuLP(G):
  delta = max([G.degree(v) for v in G.nodes()])+1

  prob = LpProblem(name='Node_Coloring_by_PuLP', sense=LpMinimize)
  y = {k: LpVariable('y'+str(k), lowBound=0, cat='Binary') for k in range(delta)}
  x = {(i,k): LpVariable('x'+str(i)+str(',')+str(k), lowBound=0, cat='Binary')
          for i in G.nodes() for k in range(delta)}

  prob += lpSum(y[k] for k in range(delta))

  for i,k in itertools.product(G.nodes(), range(delta)):
    prob += x[i,k] <= y[k]
  for (i,j), k in itertools.product(G.edges(), range(delta)):
    prob += x[i,k]+x[j,k] <= 1
  for i in G.nodes():
    prob += lpSum(x[i,k] for k in range(delta)) == 1

  prob.solve()
  if LpStatus[prob.status] == 'Optimal':
    print('Node '+str(int(value(prob.objective)))+' coloring found.')
  else:
    print("Error:")

In [12]:
node_coloring_by_PuLP(G)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/16db0967c50e4519ae961abe14b4fdbd-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/16db0967c50e4519ae961abe14b4fdbd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 58 COLUMNS
At line 227 RHS
At line 281 BOUNDS
At line 306 ENDATA
Problem MODEL has 53 rows, 24 columns and 116 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 24 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 25 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tig

In [13]:
import random
random.seed(1)
G = nx.random_geometric_graph(50,0.3)
print('Number of nodes: ',len(G.nodes()))
print('Number of edges: ',len(G.edges()))
print('Welsh_Powell: ', end=''); welsh_powell(G)
print('By PuLP: ',end=''); node_coloring_by_PuLP(G)

Number of nodes:  50
Number of edges:  251
Welsh_Powell: Node 9 coloring found.
By PuLP: Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/22fc413df1044f219f518c9ca0a9e72d-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/22fc413df1044f219f518c9ca0a9e72d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6075 COLUMNS
At line 21176 RHS
At line 27247 BOUNDS
At line 28268 ENDATA
Problem MODEL has 6070 rows, 1020 columns and 13040 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 5687 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 4980 strengthened rows, 0 substi